Objective: Develop a method to detect malicious websites from its DOM using Graph Neural Networks (GNN) to effectively utilize structural information in HTML.

Approach: We will represent HTML documents as graphs based on their Document Object Model (DOM) structure. Local features of each node, including
node attributes, will be extracted using Recurrent Neural Networks (RNN) or
LSTM. Then, we will utilize GNN [4], graph convolutional network (GCN) [5]
specifically, to model the relationships between nodes, capturing structural information within the HTML document. The final graph-level representation will
be fed into a fully connected layer for classification together with the URL data,
embedded using RNN. This approach may also be easily extended to multilabel
classification by using a sigmoid activation function in the output layer.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import Counter
import json
import os
import networkx as nx
from torch_geometric.data import Data
from pyvis.network import Network
from datasets import Dataset
import torchtext
from tqdm import tqdm

# Model

Model first takes node attributes and tags and embeds them using embedding layer and RNN. Then, it uses GCN to propagate information between nodes. Then it applies average pooling to get graph level representation and passes it through a fully connected layer for classification.

In [21]:
from torch_geometric.nn import global_mean_pool 

class GCN(nn.Module):
    def __init__(self, hidden_channels, num_classes, tag_vocab, attr_vocab):
        super(GCN, self).__init__()
        self.debug, self.logged_once = False, False
        self.tag_vocab = tag_vocab
        self.attr_vocab = attr_vocab
        self.embedding_dim_tag = 16
        self.embedding_dim_attr = 32
        self.hidden_channels = hidden_channels
        self.gcn_hidden_channels = hidden_channels + self.embedding_dim_tag
        self.num_classes = num_classes

        self.tag_embedding = nn.Embedding(len(tag_vocab['vocab']), self.embedding_dim_tag)
        self.attr_embedding = nn.Embedding(len(attr_vocab['vocab']), self.embedding_dim_attr)
        self.rnn = nn.LSTM(self.embedding_dim_attr, self.hidden_channels, batch_first=True)
        self.conv1 = GCNConv(self.gcn_hidden_channels, self.gcn_hidden_channels)
        self.conv2 = GCNConv(self.gcn_hidden_channels, self.gcn_hidden_channels)
        self.fc = nn.Linear(self.gcn_hidden_channels, self.num_classes)

    def log_shape(self, name, x):
        if self.debug and not self.logged_once:
            print(f'{name} shape: {x.shape}')

    def forward(self, tags, attrs, edge_index, batch):
        # tags: [num nodes]
        # attrs: [num nodes, seq len (128)]
        # edge_index: [2, num edges]

        # embed tags and attributes
        tag_embeds = self.tag_embedding(tags) # [num nodes, embedding dim]
        attr_embeds = self.attr_embedding(attrs) # [num nodes, seq len, embedding dim]
        
        self.log_shape('tag_embeds', tag_embeds)
        self.log_shape('attr_embeds', attr_embeds)

        # rnn
        _, hidden = self.rnn(attr_embeds) # [1, num nodes, hidden channels]
        hidden = hidden[0].squeeze(0) # [num nodes, hidden channels]
        self.log_shape('hidden2', hidden)

        # concatenate tag and attribute embeddings
        x = torch.cat([tag_embeds, hidden], dim=1) # [num nodes, hidden channels + embedding dim] 
        self.log_shape('concatenated', x)
        self.log_shape('edge_index', edge_index)

        # propagate graph convolutional neural network
        x = self.conv1(x, edge_index) # [num nodes, hidden channels + embedding dim]
        self.log_shape('conv1', x)
        x = F.relu(x)
        x = self.conv2(x, edge_index) # [num nodes, hidden channels + embedding dim]
        self.log_shape('conv2', x)
        x = F.relu(x)

        # global pooling
        x = global_mean_pool(x, batch) # [batch size, hidden channels + embedding dim]
        self.log_shape('global_pooling', x)

        # fully connected layer
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.fc(x) # [batch size, num classes]
        self.log_shape('fc', x)

        if self.debug:
            self.logged_once = True
            
        return x

# Train

In [3]:
from torch_geometric.data import InMemoryDataset

class CustomDataset(InMemoryDataset):
    def __init__(self, root, split, transform=None, pre_transform=None):
        self.split = split
        super().__init__(root, transform, pre_transform)
        self.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return [f'{self.split}_graph_data.pt']

    def download(self):
        pass

    def process(self):
        pass

In [4]:
train_dataset = CustomDataset(root='./data/preprocessed_data', split='train')
test_dataset = CustomDataset(root='./data/preprocessed_data', split='test')

In [5]:
import pickle


with open('./data/preprocessed_data/attr_vocab.pkl', 'rb') as f:
    attr_vocab = pickle.load(f)
with open('./data/preprocessed_data/tag_vocab.pkl', 'rb') as f:
    tag_vocab = pickle.load(f)
print('Vocab loaded from disk.')

Vocab loaded from disk.


In [6]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

def get_data_loader(dataset, batch_size=16):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
train_loader, test_loader = get_data_loader(train_dataset), get_data_loader(test_dataset)
print(len(train_loader), len(test_loader))

for batch in train_loader:
    print(batch)
    break

683 293
DataBatch(edge_index=[2, 18418], y=[16], tags=[9225], attrs=[9225, 128], num_nodes=9225, batch=[9225], ptr=[17])


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score, balanced_accuracy_score
from tqdm import tqdm
import numpy as np
import time
import os

# Create the directories for checkpoints
checkpoint_dir = "./checkpoints_5e4"
os.makedirs(checkpoint_dir, exist_ok=True)

# Set up tensorboard writer
writer = SummaryWriter(log_dir="./runs/gnn_experiment_5e4")

# Define checkpoint interval
checkpoint_interval = 5

# Modified training epoch function with TQDM progress bar
def train_epoch(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    epoch_loss = 0
    y_true, y_pred = [], []
    
    with tqdm(total=len(train_loader), desc="    Training", leave=True) as pbar:
        for batch_idx, data in enumerate(train_loader):
            tags, attrs, edge_index, y, batch = data.tags.to(device), data.attrs.to(device), data.edge_index.to(device), data.y.to(device), data.batch.to(device)
            optimizer.zero_grad()
            output = model(tags, attrs, edge_index, batch)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            
            y_true.append(data.y)
            y_pred.append(output.argmax(dim=1))
            epoch_loss += loss.item()
            
            writer.add_scalar('Train/Loss_per_batch', loss.item(), epoch * len(train_loader) + batch_idx)

            pbar.update(1)
    
    y_true = torch.cat(y_true, dim=0).cpu().numpy()
    y_pred = torch.cat(y_pred, dim=0).cpu().numpy()
    train_acc = balanced_accuracy_score(y_true, y_pred)
    train_f1 = f1_score(y_true, y_pred, average='weighted')
    return epoch_loss / len(train_loader), train_acc, train_f1

# Modified evaluation function with TQDM progress bar
def evaluate(model, device, data_loader, criterion, epoch):
    model.eval()
    y_true, y_pred = [], []
    losses = []
    
    with tqdm(total=len(data_loader), desc="    Testing", leave=True) as pbar:
        for batch_idx, data in enumerate(data_loader):
            tags, attrs, edge_index, y, batch = data.tags.to(device), data.attrs.to(device), data.edge_index.to(device), data.y.to(device), data.batch.to(device)
            with torch.no_grad():
                output = model(tags, attrs, edge_index, batch)
            y_true.append(data.y)
            y_pred.append(output.argmax(dim=1))
            loss = criterion(output, y).item()
            losses.append(loss)
            
            writer.add_scalar('Test/Loss_per_batch', loss, epoch * len(data_loader) + batch_idx)

            pbar.update(1)
    
    y_true = torch.cat(y_true, dim=0).cpu().numpy()
    y_pred = torch.cat(y_pred, dim=0).cpu().numpy()
    test_acc = balanced_accuracy_score(y_true, y_pred)
    test_f1 = f1_score(y_true, y_pred, average='weighted')
    return test_acc, test_f1, np.mean(losses)

# Modified training function with checkpointing and TQDM progress bar
def train(model, device, train_loader, test_loader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        start_time = time.time()
        
        print(f"Epoch {epoch + 1}:")

        # Training Phase
        train_loss, train_acc, train_f1 = train_epoch(model, device, train_loader, optimizer, criterion, epoch)
        writer.add_scalar('Train/Loss', train_loss, epoch)
        writer.add_scalar('Train/BalancedAccuracy', train_acc, epoch)
        writer.add_scalar('Train/F1Weighted', train_f1, epoch)

        # Testing Phase
        test_acc, test_f1, test_loss = evaluate(model, device, test_loader, criterion, epoch)
        writer.add_scalar('Test/Loss', test_loss, epoch)
        writer.add_scalar('Test/BalancedAccuracy', test_acc, epoch)
        writer.add_scalar('Test/F1Weighted', test_f1, epoch)

        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Time: {elapsed_time:.2f} s | Train Loss: {train_loss:.2f} | Train Acc: {train_acc:.2f} | Train F1: {train_f1:.2f} | "
              f"Test Loss: {test_loss:.2f} | Test Acc: {test_acc:.2f} | Test F1: {test_f1:.2f}\n")

        # Save checkpoint every 'checkpoint_interval' epochs
        if (epoch + 1) % checkpoint_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}.pt")
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, checkpoint_path)

In [29]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(hidden_channels=32, num_classes=2, tag_vocab=tag_vocab, attr_vocab=attr_vocab).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()
train(model, device, train_loader, test_loader, optimizer, criterion, epochs=10000)

# Close the tensorboard writer
writer.close()

Epoch 1:


    Testing: 100%|███████████████████████████████████████████████████████████████████| 293/293 [00:04<00:00, 62.32it/s]


Time: 139.10 s | Train Loss: 0.63 | Train Acc: 0.56 | Train F1: 0.58 | Test Loss: 0.61 | Test Acc: 0.59 | Test F1: 0.60

Epoch 2:


    Testing: 100%|███████████████████████████████████████████████████████████████████| 293/293 [00:04<00:00, 62.13it/s]


Time: 47.09 s | Train Loss: 0.58 | Train Acc: 0.65 | Train F1: 0.68 | Test Loss: 0.52 | Test Acc: 0.76 | Test F1: 0.78

Epoch 3:


...

Time: 226.76 s | Train Loss: 0.00 | Train Acc: 1.00 | Train F1: 1.00 | Test Loss: 0.93 | Test Acc: 0.92 | Test F1: 0.92

Epoch 235:


    Testing: 100%|███████████████████████████████████████████████████████████████████| 293/293 [00:17<00:00, 16.46it/s]


Time: 228.93 s | Train Loss: 0.00 | Train Acc: 1.00 | Train F1: 1.00 | Test Loss: 1.01 | Test Acc: 0.92 | Test F1: 0.92

Epoch 236:


    Training:  15%|█████████▉                                                        | 103/683 [00:33<03:06,  3.10it/s]


KeyboardInterrupt: 

In [27]:
torch.cuda.empty_cache()
writer.close()

In [25]:
def test_model(model, device, data_loader):
    model.eval()
    y_true, y_pred = [], []
    for data in data_loader:
        tags, attrs, edge_index, y, batch = data.tags.to(device), data.attrs.to(device), data.edge_index.to(device), data.y.to(device), data.batch.to(device)
        with torch.no_grad():
            output = model(tags, attrs, edge_index, batch)
        y_true.append(data.y)
        y_pred.append(output.argmax(dim=1))
    y_true = torch.cat(y_true, dim=0).cpu().numpy()
    y_pred = torch.cat(y_pred, dim=0).cpu().numpy()
    return y_true, y_pred

In [26]:
y_true, y_pred = test_model(model, device, test_loader)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))
print()
print(f'Balanced Accuracy: {balanced_accuracy_score(y_true, y_pred)}')
print(f'F1 Score: {f1_score(y_true, y_pred)}')

[[2551  247]
 [ 203 1678]]
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2798
           1       0.87      0.89      0.88      1881

    accuracy                           0.90      4679
   macro avg       0.90      0.90      0.90      4679
weighted avg       0.90      0.90      0.90      4679


Balanced Accuracy: 0.9019006702972694
F1 Score: 0.8817656332107199


In [18]:
class RandomModel(nn.Module):
    def __init__(self, num_classes):
        super(RandomModel, self).__init__()
        self.num_classes = num_classes

    def forward(self, tags, attrs, edge_index, batch):
        batch_size = len(np.unique(batch.cpu().numpy()))
        return torch.rand((batch_size, self.num_classes)).to(device)
    
# random model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RandomModel(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()

accs, f1s, losses = [], [], []
for i in range(5):
    acc, f1, loss = evaluate(model, device, test_loader, criterion, desc="Testing")
    accs.append(acc)
    f1s.append(f1)
    losses.append(loss)
print(f'Random Model | Test Accuracy: {np.mean(accs)} | Test F1: {np.mean(f1s)} | Test Loss: {np.mean(losses)}')

    Testing: 100%|██████████████████████████████████████████████████████████████████| 293/293 [00:01<00:00, 185.56it/s]

Random Model | Test Accuracy: 0.4997196676140283 | Test F1: 0.5043534997251669 | Test Loss: 0.7143093726740762
